In [812]:
from openpyxl import load_workbook

In [813]:
excel_path = "scheme.xlsx"
sheet_name = "Sheet1"
latex_path = "scheme.tex"

In [814]:
def escape(s: str):
    return s.replace("_", "\\_")

In [815]:
def table_to_latex(filename, sheet_name, headers: str | None=None):
    workbook = load_workbook(filename)
    sheet = workbook[sheet_name]
    latex = ""

    if not headers:
        # latex += "\\begin{tabular}{" + "l" * sheet.max_column + "}\n"
        latex += "\\begin{tabularx}{\\textwidth}{" + "X" * sheet.max_column + "}\n"
    else:
        # latex += "\\begin{tabular}{" + headers + "}\n"
        latex += "\\begin{tabularx}{\\textwidth}{" + headers + "}\n"

    latex += "\\toprule\n"

    for row in sheet.iter_rows():
        is_last_row = True
        cells_latex = []
        cline_ranges = []

        for cell in row:
            is_last_row = cell.row == sheet.max_row

            # Find merged range
            is_merged, is_merged_master, is_merged_tail = False, False, False
            for merged_range in sheet.merged_cells.ranges:
                if cell.coordinate in merged_range:
                    is_merged = True
                    is_merged_master = is_merged and cell.value is not None
                    is_merged_tail = is_merged and cell.column == merged_range.max_col and cell.row == merged_range.max_row
                    break

            cell_value = escape(str(cell.value)) if cell.value else ""
            cell_value = f"\\textbf{{{cell_value}}}" if cell.font.bold else cell_value
            cell_value = f"\\underline{{{cell_value}}}" if cell.font.underline else cell_value
            cell_value = f"\\textit{{{cell_value}}}" if cell.font.italic else cell_value

            if is_merged:
                merged_size_columns = merged_range.max_col - merged_range.min_col + 1
                merged_size_rows = merged_range.max_row - merged_range.min_row + 1

                print(cell.value, merged_size_columns, merged_size_rows)

                if is_merged_master:
                    l = cell_value
                    if merged_size_columns > 1:
                        l = f"\\multicolumn{{{merged_size_columns}}}{{l}}{{{l}}}"
                    if merged_size_rows > 1:
                        l = f"\\multirow{{{merged_size_rows}}}[{int(merged_size_rows * 2)}]{{*}}{{{l}}}"
                    cells_latex.append(l)

                elif merged_size_rows > 1:
                    cells_latex.append("")

            else:
                cells_latex.append(f"{cell_value}")

            if not is_merged:
                cline_ranges.append(cell.column)
            elif is_merged_tail:
                cline_ranges.append(cell.column)
            elif is_merged and merged_size_columns > 1 and merged_size_rows == 1:
                cline_ranges.append(cell.column)

        latex += " & ".join(cells_latex) + " \\\\\n"

        if not is_last_row:
            latex += "".join(
                map(lambda x: f"\\cmidrule{{{str(x)}-{str(x)}}}", set(cline_ranges))
            )
            latex += "\n"

    latex += "\\bottomrule\n"
    latex += "\\end{tabularx}\n\n"

    return latex

In [816]:
latex = ""
latex += table_to_latex(excel_path, "Sheet1", headers="p{1.8cm}llX")
latex += table_to_latex(excel_path, "Sheet2", headers="p{1.8cm}llXl")

Entity 1 10
Module 1 3
None 1 10
None 1 3
None 1 10
None 1 3
None 1 10
(name, qualified_name and file_path) 2 1
None 2 1
None 1 10
Function 1 2
(name, qualified_name and file_path) 2 1
None 2 1
None 1 10
None 1 2
None 1 10
Method (Function) 1 2
(name, qualified_name and file_path) 2 1
None 2 1
None 1 10
None 1 2
None 1 10
Variable 1 2
(name, qualified_name and file_path) 2 1
None 2 1
None 1 10
None 1 2
Relationship 1 16
None 1 16
None 1 7
None 1 16
Containment 1 6
CONTAINS 1 6
None 1 7
None 1 16
None 1 6
None 1 6
None 1 7
None 1 16
None 1 6
None 1 6
None 1 7
None 1 16
None 1 6
None 1 6
None 1 7
None 1 16
None 1 6
None 1 6
None 1 7
None 1 16
None 1 6
None 1 6
None 1 7
None 1 16
Parameter 1 2
TAKES 1 2
Function TAKES Class 1 2
None 1 16
None 1 2
None 1 2
None 1 2
None 1 16
None 1 6
None 1 16
None 1 6
None 1 16
None 1 6
None 1 16
Utilization 1 3
USES 1 3
None 1 6
None 1 16
None 1 3
None 1 3
None 1 6
None 1 16
None 1 3
None 1 3
None 1 6


In [817]:
# Change font for keywords
keywords = [
    "Module",
    "Class",
    "Function",
    "Method",
    "Variable",
    "qualified_name",
    "name",
    "file_path",
    "access",
    "is_abstract",
    "CALLS",
    "INHERITS",
    "CONTAINS",
    "TAKES",
    "RETURNS",
    "YIELDS",
    "INSTANTIATES",
    "USES",
    "subtype",
    "public",
    "private",
    "protected",
    "function",
    "staticmethod",
    "classmethod",
    "method",
    "param_name",
    "default_value",
    "arguments",
]

for keyword in keywords:
    latex = latex.replace(escape(keyword), f"\\textsf{{{escape(keyword)}}}")

In [818]:
# Save the LaTeX code to a .tex file
with open(latex_path, "w") as f:
    f.write(latex)